# Collect the Simulated Retrospective Reanalysis Data of the National Water Model

This jupyter notebook downloads the National Water Model (NWM) retrospective reanalysis data. The NWM reanalysis data are large NetCDF files and are stored within a scratch directory. The ouput directory contains a csv file that shows NWM grid indices associated with each SNOTEL gage. This csv file will be used later in other scripts to retrieve different variables (such as snow water equivalent, snow covered area, ...) at SNOTEL locations.

## 1. Import Libraries
In this section, required python libraries are installed and imported. 

In [ ]:
# Import libraries

import pandas as pd 
import os
import numpy as np
import netCDF4        # pip install netCDF4 
import xarray as xr   # pip install xarray 
import pyproj         # pip install pyproj 
from datetime import datetime

## 2.  Read SNOTEL Information and Define Parameters


In [ ]:
input_dir = '../input'                                           # The path to the CSV file including the SNOTEL sites information
output_dir = '../output'                                         # The path to save CSV files including indices, SWE, and Precipitation values. 

dataset = 'LDASOUT'                                              # The type of the NWM output.  LDASOUT is the output of the land surface model.
st = '2007-01-01'                                                # Start date of the period of interest
et = '2007-01-02'                                                # End date of the period of interest
Glist = 'GoogleList.txt'                                         # The name of a file including full path to the NWM data
data_dir='/oasis/projects/nsf/usu104/igarousi/projects/scratch'  # The path to download the NWM data

nrcs_file = "NRCS_SNOTEL_no_Alaska_Joint_w_CEC.csv"              # This is an input. A csv file that includes information about SNOTEL sites
nwm_file = "201801010300.LDASOUT_DOMAIN1.comp"                   # This is an input. A netcdf file that is the result of land surface model in NWM.
variable = 'SNEQV'                                               # The variable of interest as used in the NWM
output_name = 'SNOTEL_indices_at_NWM.csv'                        # The name for output in Get_NWM_indices function

## 3.  Generate a List of Full Paths to NWM Data
The following cell will create a list of `gs://national-water-model-v2/full_physics/*` into 'GoogleList.txt' file.  

In [ ]:
dates = pd.date_range(st, et, freq='1H')

# Create full paths and append to Gname list
Gname = []
for d in dates:

    google_name = f'gs://national-water-model-v2/full_physics/{d.year}/{d.year}{str(d.month).zfill(2)}{str(d.day).zfill(2)}{str(d.hour).zfill(2)}{str(d.minute).zfill(2)}.{dataset}_DOMAIN1.comp'
    Gname.append(google_name)

# Save as a text file  
with open(os.path.join(output_dir, Glist), 'w') as f:
    for i in Gname:
        if i != Gname[-1]:   # this is because I don't want a new empty line at the end of the file
            f.write(i+'\n')
        else:
            f.write(i)

## 4.  Download the NWM Data Using `gsutil` Library

The gsutil library needs to be installed if it is not installed yet. The following batch script installs gsutil. Note that the file should exist in code directory.


In [ ]:
!./gsutil_install.sh

The following command save results (i.e., downloaded netcdf files) at a scratch directory that exist on the top-level directory. The size of files are large and you need to make sure that you have enough storage. I have ~ 1.5 TB storage on my project directory. This is not enought becasue for 12 year of data I need ~ 8.5 TB. So, what I do is to choose small periods in the second cell above, download data, and then move them to the comet scratch directory where I have more storage to save files temporarily. After the analysis is done, I will delet these files. **Question**: Why don't I save them on comet scratch directory the first place? Because, I get OSError: Read-only file system error! 

Note that the temporal resolution of the land surface model outputs for retrospective configuration is 3-hour.  That is why we get "No URLs matched ..." message for several hours after running the following cell.

In [ ]:
!./gsutil_run.sh

Make sure to move files to `/oasis/scratch/comet/igarousi/temp_project` becasue this path will be used in the next scripts.

## 5.  Define a Function to Get Indices

The following function takes SNOTEL sites information and the projection coordinate system from a NWM file as inputs and retrives X and Y indices for each SNOTEL site. The output of this function is a csv file that includes information of NWM indices for each SNOTEL site. This will be used as an input to the Get_NWM_values function.

In [ ]:
def Get_NWM_indices(input_dir, nrcs_file, nwm_file, output_dir, output_name):

    '''
    input_dir   : Input path where the nrcs_file (snotel information) and NWM (ldasout example) exist. 
    nrcs_file   : A csv file that includes information about SNOTEL sites.
    nwm_file    : A netcdf file that is the result of land surface model in NWM.
    Y_NWM       : An array holding Y indices of the NWM grid cells.
    X_NWM       : An array holding X indices of NWM grid cells.
    output_dir : Path to save a csv file as the output of this function. 
                  It incldues useful information about SNOTEL sites most importantly the associated 
                  indices that are used to retrieve NWM variales later. 
    '''
    
    Data=pd.DataFrame([])
    
    # Use the site information and generate a dataframe 
    nrcs = pd.read_csv(os.path.join(input_dir, nrcs_file))
    
    # Open the example NWM file and read the projection. 
    # Then, create a projection using pyproj libarary and using 'crs' information from the above object
    nwm_example = netCDF4.Dataset(os.path.join(input_dir, nwm_file))
    pr = pyproj.Proj(nwm_example.variables['crs'].esri_pe_string)
    
    # Open the example NWM file and read  X and Y indices of all grids from one NWM file
    nc_NWM = xr.open_dataset(os.path.join(input_dir, nwm_file))
    X_NWM = nc_NWM.coords['x']
    Y_NWM = nc_NWM.coords['y']
    
    # Loop over the sites
    for i in range(0, len(nrcs['Station_ID'])):
        
        # Generate projected values (proj_y and proj_x) from latitude and longitude of SNOTEL gages 
        # Note: in pr, the first element should be longitude and the second is latitude
        # Note: results of pr, the first element is x and the second is y
        proj_x, proj_y = pr(nrcs['Longitude'][i], nrcs['Latitude'][i])
        
        # Calculate the distance between the center of grids from the location of the site
        distance = ((Y_NWM - proj_y)**2 + (X_NWM - proj_x)**2)**0.5
        yindex, xindex = np.where(distance == distance.min())
                                  
        # Create a dataframe
        data = pd.DataFrame({'col1': nrcs['Station_ID'][i],
                             'col2': nrcs['Station_Na'][i], 
                             'col3': nrcs['NAME'][i],
                             'col4': nrcs['Longitude'][i], 
                             'col5': nrcs['Latitude'][i],
                             'col6': proj_x, 
                             'col7': proj_y, 
                             'col8': xindex, 
                             'col9': yindex})
                                  
        # create a dataframe
        frames = [Data, data]
        Data = pd.concat(frames, ignore_index=True)
        
                                  
    # Save dataframe as a csv file
    Data.columns = ['Station_ID', 'Station_Name', 'Ecoregion_NAME', 'Longitude', 'Latitude', 'proj_x', 'proj_y', 'Xindex', 'Yindex']
    Data.to_csv(os.path.join(output_dir, output_name), index=False)
                                  
    
    return Data

## 6.  Run the Function

Retrieve location indices and show available variales

In [ ]:
Site_Info = Get_NWM_indices(input_dir, nrcs_file, nwm_file, output_dir, output_name)